<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/Project/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, output
from IPython.display import HTML, Javascript
from IPython.display import clear_output
from firebase_admin import credentials
from firebase_admin import firestore
import firebase_admin
import IPython
import random
import json
import os

directory = '/content/CloudComputing-tirgul'

if os.path.exists(directory):
    !rm -r "$directory"

os.chdir('/content')  # Change current directory to /content
!git clone "https://github.com/ndvp39/CloudComputing-tirgul"

# drive.mount('/content/drive')

Cloning into 'CloudComputing-tirgul'...
remote: Enumerating objects: 4216, done.
remote: Counting objects: 100% (957/957), done.
remote: Compressing objects: 100% (497/497), done.
remote: Total 4216 (delta 654), reused 683 (delta 435), pack-reused 3259
Receiving objects: 100% (4216/4216), 20.27 MiB | 11.83 MiB/s, done.
Resolving deltas: 100% (1648/1648), done.


In [2]:
!pip install requests beautifulsoup4

import requests
from bs4 import BeautifulSoup

import nltk
from nltk.chat.util import Chat, reflections

In [3]:
%cd /content/CloudComputing-tirgul/Project/Index/

%run index.ipynb

/content/CloudComputing-tirgul/Project/Index


Get index from db

In [4]:
# GET INDEX FROM DB
index = get_index_from_db()

ChatBot

In [5]:
%cd /content/CloudComputing-tirgul/Project/Chatbot/
%run chatbot.ipynb

/content/CloudComputing-tirgul/Project/Chatbot
/content/CloudComputing-tirgul/Project/Index


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
#saveIndex(index)
db_collection_name = "ChatBotPatterns"
chatbot_patterns = get_patterns_from_db(db_collection_name)

# Create a chatbot
chatbot = Chat(chatbot_patterns, reflections)


SearchEngine

In [7]:
# Load and display a web page.
# Supports two types of HTML docs - 1. combined with css and js | 2. 3 separate files
def switchToScreen(css_filename_, html_filename_, js_filename_):
    clear_output(wait=True)

    if css_filename_ is None and js_filename_ is None:
        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))
    else:
        css_response = requests.get(css_filename_)
        css_content_ = css_response.text
        display(HTML(f'<style>{css_content_}</style>'))

        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))

        js_response = requests.get(js_filename_)
        js_content_ = js_response.text
        display(Javascript(js_content_))

In [8]:


def get_title_from_link(link):

  try:
        # Make an HTTP request
        response = requests.get(link)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the <h1> tag
            h1_tag = soup.find('h1')
            if h1_tag:
              return h1_tag.get_text()
            else:
              h2_tag = soup.find('h2')
              if h2_tag:
                return h2_tag.get_text()
        else:
            return f"Failed to fetch the page. Status code: {response.status_code}"

  except Exception as e:
        return f"An error occurred: {e}"

def check_title_list(lst):
  seen = {}
  result = []
  for word in lst:
    if word not in seen:
      seen[word] = 1
      result.append(word)
    else:
      seen[word] += 1
      result.append(f"{word}_{seen[word] - 1}")
  return result


def get_titles_links_of_search(search_string):
  query_counter = 0
  titles = list()
  links = set()
  search_string = search_string.split()
  # trimming and steaming algorithem to search terms exactly as they are in the DB
  search_string = remove_stop_words(search_string)
  search_string = trim_words(search_string)
  search_string = apply_stemming(search_string)
  for word in index.keys():
    if word in search_string:
      for i in range(min(10, len(index[word]))):
        curr_link = getLinkFromString(word, i)
        query_counter += index[word][curr_link]['counter']
        if curr_link not in links:
          titles.append(get_title_from_link(curr_link))
        links.add(curr_link)
  titles = check_title_list(titles)
  return titles, links, query_counter






In [9]:
def receive_search_string(search_string):
  original_search_string = search_string
  search_string = search_string.lower()
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/results.html"
  js_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/script.js"
  switchToScreen(css_filename, html_filename, js_filename)

  js_code = f"""
    <script>
      document.getElementById("searchbartext").value = "{original_search_string}";
    </script>
    """

  titles, links, times_appeared = get_titles_links_of_search(search_string)

  # Convert Python arrays to JavaScript arrays
  titles_js_array = ', '.join(f'"{title}"' for title in titles)
  links_js_array = ', '.join(f'"{link}"' for link in links)
  times_appeared_str = str(times_appeared)

  js_code_2 = f"""
    <script>
      renderResults_title_link([ {titles_js_array} ], [ {links_js_array} ], [{times_appeared_str}]);
    </script>
  """

  display(HTML(js_code))
  display(HTML(js_code_2))

  if titles:
    update_or_add_statistic(search_string)

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_search_string', receive_search_string)

def open_chatbot_page():
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/chatbot.html"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_chatbot_page', open_chatbot_page)

def receive_question_chatbot(question):
  response = chatbot.respond(question)
  if response is None:
    response = "Sorry, i dont have response for your answer :/"
  js_code = f"""
      <script>
        ChatbotResponse("{response}");
      </script>
    """
  #print(response)
  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_question_chatbot', receive_question_chatbot)

def back_to_home_page():
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/index.html"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_home_page', back_to_home_page)

css_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/style.css"
js_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/script.js"
html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/index.html"

switchToScreen(css_filename, html_filename, js_filename)


<IPython.core.display.Javascript object>

In [10]:
def open_indexeditor_page():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_indexeditor_page', open_indexeditor_page)

def open_statistics_screen():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/statistics.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_statistics_screen', open_statistics_screen)

def back_to_admin():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_admin', back_to_admin)


In [11]:
def switchToScreen(css_filename_, html_filename_, js_filename_):
    clear_output(wait=True)

    if css_filename_ is None and js_filename_ is None:
        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))
    else:
        css_response = requests.get(css_filename_)
        css_content_ = css_response.text
        display(HTML(f'<style>{css_content_}</style>'))

        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))

        js_response = requests.get(js_filename_)
        js_content_ = js_response.text
        display(Javascript(js_content_))

Administration:

In [12]:
%cd /content/CloudComputing-tirgul/Project/Administration/
%run administration.ipynb

/content/CloudComputing-tirgul/Project/Administration
